In [ ]:
# ==================================================================================================
# Yufei Li
# Princeton University
# yl5385@princeton.edu

# Feburay 2023

# Note:In this demo, the neural network is synthesized using the TensorFlow (verion: 2.11.0) framework. 
# Please install TensorFlow according to the official guidance, then import TensorFlow and other dependent modules.
# ==================================================================================================

!pip install pandas numpy matplotlib
!pip install tensorflow

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import copy
import csv
import math
import cmath
import time

inFilename = "Input_Yo20op1084_ana.csv"
outFilename = "Output_Yo20op1084_ana.csv"

Input = pd.read_csv(inFilename,header=None)
Output = pd.read_csv(outFilename,header=None)

print(Input.dtypes)
print(Output.dtypes)

inputs = []
outputs = []

inputs = np.array(Input)
outputs = np.array(Output)
print(inputs)
print(outputs)

In [ ]:
# Randomize the order of the inputs, so they can be evenly distributed for training, testing, and validation

num_inputs = len(inputs)
print("Total Number of Dataset is:",num_inputs)
randomize = np.arange(num_inputs)
print(randomize)

inputs_origin = copy.deepcopy(inputs[randomize])
outputs_origin = copy.deepcopy(outputs[randomize])
print(inputs_origin)

random.Random(5).shuffle(randomize)
print(randomize)
# Swap the consecutive indexes (0, 1, 2, etc) with the randomized indexes
inputs_real = copy.deepcopy(inputs_origin[randomize])
outputs_real = copy.deepcopy(outputs_origin[randomize])
print(inputs_real)
print(outputs_real)

# Split the recordings (group of samples) into two sets: training and testing
TRAIN_SPLIT = int(0.7 * num_inputs)
inputs_train, inputs_test = np.split(inputs_real, [TRAIN_SPLIT])
outputs_train, outputs_test = np.split(outputs_real, [TRAIN_SPLIT])

num_inputs_train = len(inputs_train)
print("Total Number of training Dataset is:",num_inputs_train)
print("Dataset randomization and separation complete!")

In [ ]:
# build the model and train it
initial_learning_rate = 0.020669824682365133
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
  initial_learning_rate,
  decay_steps=375*65,
  decay_rate=0.575401007701697,
  staircase=True)

Normlayer1=tf.keras.layers.Normalization()
Normlayer1.adapt(inputs_real)

model = tf.keras.Sequential()
model.add(Normlayer1)
model.add(tf.keras.layers.Dense(43, activation='sigmoid'))
model.add(tf.keras.layers.Dense(56, activation='sigmoid'))
model.add(tf.keras.layers.Dense(43, activation='sigmoid'))
model.add(tf.keras.layers.Dense(8))

#Train using Adam
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=opt, loss='mse', metrics=['mse'])

t2_start = time.perf_counter()
history = model.fit(inputs_train, outputs_train, epochs=600, batch_size=16, validation_data=(inputs_test, outputs_test))
t2_stop = time.perf_counter()
print("Elapsed time: ", t2_stop - t2_start)

print(model.evaluate(inputs_test, outputs_test, verbose=0))

In [ ]:
# # increase the size of the graphs. The default size is (6,4).
plt.rcParams["figure.figsize"] = (9,6)

# summarize history for loss
plt.plot(history.history['loss'], '-r.', label='Training Loss')
plt.plot(history.history['val_loss'], '--b.', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.semilogy()
plt.grid()
plt.show()

print(plt.rcParams["figure.figsize"])
#print(history.history)

In [ ]:
# Save inputs index
with open('Yo20op1084_ana_inputs.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(Y_input)
# Save predcition from FNN
with open('Yo20op1084_ana_prediction.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(Y_pred)
# Save original data
with open('Yo20op1084_ana_outputs.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(Y_output)

In [ ]:
# Use the model to predict test data
output_pred = model.predict(inputs_test)
#print(output_pred)
output_diff = output_pred - outputs_test
MSE = np.square(np.subtract(output_pred,outputs_test)).mean()
print(output_diff)
print(MSE)